<a href="https://colab.research.google.com/github/kwen1510/testimonial_writer/blob/main/Testimonial_Writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Testimonial Writer
Run through the cells one by one.

In [ ]:
# Install relevant libraries
!pip install happytransformer
!pip install transformers sentencepiece sacremoses
!pip install python-docx

In [ ]:
from happytransformer import HappyGeneration
gpt_neo = HappyGeneration(model_type="GPT-NEO", model_name="EleutherAI/gpt-neo-125M") 

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/526M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [11]:
# https://towardsdatascience.com/different-ways-to-connect-google-drive-to-a-google-colab-notebook-pt-1-de03433d2f7a

from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive/Testimonial_Writer
!ls

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Testimonial_Writer
doc  train.txt	txt


In [ ]:
from happytransformer import GENSettings

seed_phrase = "Gabriel is a kind and respectful student."

top_k_sampling_settings = GENSettings(do_sample=True, top_k=50, max_length=30, min_length=10)
output_top_k_sampling = gpt_neo.generate_text(seed_phrase, args=top_k_sampling_settings)

subsequent_text = output_top_k_sampling.text
full_text = seed_phrase + subsequent_text

print(full_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Gabriel is a kind and respectful student. She is dedicated to helping all those who need her, who want to help others see them as they are, even if only as a kind, compassionate


#Fine-tuning


In [ ]:
# https://happytransformer.com/learning-parameters/

# https://happytransformer.com/save-load-model/ (saving and loading weights)

from happytransformer import HappyGeneration, GENTrainArgs

gpt_neo = HappyGeneration("GPT-Neo", "EleutherAI/gpt-neo-125M") 

train_args = GENTrainArgs(num_train_epochs=3) # epoch=3

gpt_neo.train("train.txt", args=train_args)

In [ ]:
def generate_text_from_trained(seed_phrase, min_length=50, max_length=200):

  top_k_sampling_settings = GENSettings(do_sample=True, top_k=50, max_length=max_length, min_length=min_length)
  output_top_k_sampling = gpt_neo.generate_text(seed_phrase, args=top_k_sampling_settings)

  subsequent_text = output_top_k_sampling.text
  full_text = seed_phrase + subsequent_text

  return full_text

In [ ]:
# Allow text wrap

from IPython.display import HTML, display
 
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Test with a seed phrase
seed_phrase = "He is a hardworking and diligent student. He"

generated_text = generate_text_from_trained(seed_phrase,50,200)

print("")
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



He is a hardworking and diligent student. He keeps the classroom clean and maintains his composure during lessons and lessons in an amiable and confident manner. He is diligent and meticulous as well, and this allows him to be a valuable class member in whichever organisation he is from. He is also a kind and good friend to both his teachers and peers. Possessing a sincere and kind heart, he will always find a good fit for his job in any organisation or organisation. He is a responsible individual who contributes to the community - he displays commitment to all that he undertakes, and he enjoys interacting with people of all social backgrounds. As a student in the History and Economics department, he made a conscious effort to understand the issues that led to the development of the country. He was receptive to feedback, was mature and was highly disciplined. When facing adversity, he was especially passionate about making a positive impact, and he developed a positive attitude even a

# Create paraphraser here
Looks like I might not actually need it. And it looks more like a summary than a paraphraser.

In [ ]:
# Import transformers

from transformers import *

In [ ]:
# models we gonna use for this tutorial
model_names = [
  "tuner007/pegasus_paraphrase",
  "Vamsi/T5_Paraphrase_Paws",
  "prithivida/parrot_paraphraser_on_T5", # Parrot
]

In [ ]:
model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
tokenizer = PegasusTokenizerFast.from_pretrained("tuner007/pegasus_paraphrase")

In [ ]:
def get_paraphrased_sentences(model, tokenizer, sentence, num_return_sequences=5, num_beams=5):
  # tokenize the text to be form of a list of token IDs
  inputs = tokenizer([sentence], truncation=True, padding="longest", return_tensors="pt")
  # generate the paraphrased sentences
  outputs = model.generate(
    **inputs,
    num_beams=num_beams,
    num_return_sequences=num_return_sequences,
  )
  # decode the generated sentences using the tokenizer to get them back to text
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [ ]:
# Split the passage up into sentences, then combine back together

passage = "Jane is a House Captain for the Moor-Tarbet House Directorate of Students' Council. In her Co-curricular Activity (CCA), her role was to oversee the entire CCA and made sure no one is slacking. During that time, She planned a big big big big big event! From her experiences, She learnt how to manage expectations and plan excellent events."

sentences = passage.split(".")

for sentence in sentences:
  sentence = sentence + "."

  print(sentence)

  # Get first element of returned value
  paraphrased_sentence = get_paraphrased_sentences(model, tokenizer, sentence, num_beams=10, num_return_sequences=1)[0]

  print(paraphrased_sentence)

# Generate Testimonials


In [ ]:
import random

# Student profile
name = 'Jane' #@param {type:"string"}
gender = 'female' #@param ["male", "female"]

# Initialise empty paragraphs
paragraph_1 = ""
paragraph_2 = ""
paragraph_3 = ""
paragraph_4 = ""
paragraph_5 = ""
paragraph_6 = ""
paragraph_7 = ""
paragraph_8 = ""

# Create gender checking code
def check_gender(text, gender):
  if gender == 'female':
    text = text.replace(" he ", " she ")
    text = text.replace(" his ", " her ")
    text = text.replace(" him ", " her ")
    text = text.replace(" him.", " her.")
    text = text.replace("himself", "herself")
    text = text.replace("His ", "Her ")
    text = text.replace("He ", "She ")
  else:
    text = text.replace(" she ", " he ")
    text = text.replace(" her ", " his ")
    text = text.replace(" her.", " him.")
    text = text.replace("herself", "himself")
    text = text.replace("Her ", "His ")
    text = text.replace("She ", "He ")   

  return text

In [ ]:
# Paragraph 1 - General traits

#@markdown Please reset the cell to regenerate the paragraph

def a_or_an(word):
  vowels = ['a','e','i','o','u']
  if word[0] in vowels:
    return ' an '
  else:
    return ' a '

trait_1 = 'hardworking' #@param {type:"string"}
trait_2 = 'respectful' #@param {type:"string"}

seed_phrase_1 = name + " is" + a_or_an(trait_1) + trait_1
seed_phrase_2 = "He is" + a_or_an(trait_2) + trait_2

number_of_choices = 5 #@param {type:"slider", min:1, max:10, step:1}

# Create an array
generated_text_array = []

for i in range(number_of_choices):

  generated_text_1 = generate_text_from_trained(seed_phrase_1,10,30)

  # Remove text after full stop
  generated_text_1 = generated_text_1.split('.')[0] + '.'

  generated_text_2 = generate_text_from_trained(seed_phrase_2,10,30)

  # Remove text after full stop
  generated_text_2 = generated_text_2.split('.')[0] + '.'

  paragraph_1 = generated_text_1 + " " + generated_text_2

  paragraph_1 = check_gender(paragraph_1, gender)

  generated_text_array.append(paragraph_1)

for index, paragraph in enumerate(generated_text_array):
  print("")
  print(f"{index+1}: {paragraph}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



1: Jane is a hardworking and intelligent individual who consistently puts in effort to improve herself. She is a respectful character who constantly seeks to improve by giving her views regardless and her opinions often convey a positive attitude, her moral character is a testament to her intellectual strength.

2: Jane is a hardworking lady with a heart to serve the wider community. She is a respectful young lady who embraces a warm and inclusive spirit and will be an asset to any institution; her friends surely feel the same way about her.

3: Jane is a hardworking student. She is a respectful and caring individual who has the potential to excel in any area of her work.

4: Jane is a hardworking young lady with an inquiring mind. She is a respectful student who was willing to go the extra mile to help with her academic work and would willingly listen to and understand the views of others.

5: Jane is a hardworking, self-directed and focused individual and has consistently demonstrat

In [ ]:
# Can include a section on FIRE award

choice = 5 #@param {type:"integer"}

likeable = True #@param {type:"boolean"}

FIRE_award = True #@param {type:"boolean"}

if choice > number_of_choices or choice < 1:
  print(f"Please choose a number from 1 to {number_of_choices}.")
else:
  paragraph_chosen = generated_text_array[choice-1]

  # Include text if student is likeable
  if likeable:
    paragraph_chosen = paragraph_chosen + " He has an amiable personality and is well-liked by his peers."

  if FIRE_award:
    paragraph_chosen = paragraph_chosen + " He was quite exemplary in his behaviour and attitude and thus he was a recipient of the school's F.I.R.E. award - which is for students who are role models and display the school's core values of fortitude, integrity, respect and enterprise."

  paragraph_1 = paragraph_chosen

  paragraph_1 = check_gender(paragraph_1, gender)

  print(f"You chose: {paragraph_1}")

You chose: Jane is a hardworking, self-directed and focused individual and has consistently demonstrated exemplary attitude and self-motivation. She is a respectful student who actively participates in class discussions. She has an amiable personality and is well-liked by her peers. She was quite exemplary in her behaviour and attitude and thus she was a recipient of the school's F.I.R.E. award - which is for students who are role models and display the school's core values of fortitude, integrity, respect and enterprise.


In [ ]:
# Paragraph 2 - Academics

acad_passage = {
    'weak':"When it came to his academic studies, <name> set clear targets for himself and strove to achieve them. He demonstrated perseverance in the face of adversity and was never afraid to seek help when needed.",
    'okay':"Academically, <name> was a conscientious student who strove to do his best in every situation. He was persistent in his pursuit of improvement and never gave up in the face of challenges. Unyielding towards setbacks, <name>'s perseverance paid off as he made steady improvement in many of his subjects.",
    'outstanding':"Academically, <name> was a consistent and strong performer. His ability to reflect on his learning and keep pace with the rigorous school curriculum were key to his academic success. As an independent learner, <name> took a systematic and proactive approach to monitoring his own progress. He developed strong cognitive and analytical skills over the two years and impressed his teachers with his dedication to his academic responsibilities."
}

student_acad_ability = "outstanding" #@param ["weak", "okay", "outstanding"]

paragraph_2 = acad_passage[student_acad_ability]

paragraph_2 = paragraph_2.replace('<name>',name)

paragraph_2 = check_gender(paragraph_2, gender)

print(paragraph_2)

# Need to include paraphraser here

Academically, Jane was a consistent and strong performer. Her ability to reflect on her learning and keep pace with the rigorous school curriculum were key to her academic success. As an independent learner, Jane took a systematic and proactive approach to monitoring her own progress. She developed strong cognitive and analytical skills over the two years and impressed her teachers with her dedication to her academic responsibilities.


In [ ]:
# Paragraph 3 - PW

# Can include a few more standard answers

role = "member" #@param ["N.A.", "leader", "member"]

#@markdown Please check the overwrite box to key in your own comments

overwrite = True #@param {type:"boolean"}

#@markdown "As the leader/member of his Project Work group, ..."
text = "she made sure that everyone put in the due effort. She was respectful of people's opinions." #@param {type:"string"}

role_passage = {
    'leader':"As the leader of his Project Work group, ",
    'member':"As a member of his Project Work group, "
}

if role == "N.A.":
  paragraph_3 = ""
elif overwrite == False:
  paragraph_3 = role_passage[role] + "<name> was always respectful of others’ opinions and he always took the time to explain the rationale behind his decisions to ensure that everyone was comfortable and happy with the outcome."
else:
  paragraph_3 = role_passage[role] + text
  
paragraph_3 = paragraph_3.replace('<name>',name)

paragraph_3 = check_gender(paragraph_3, gender)

print(paragraph_3)

As a member of her Project Work group, she made sure that everyone put in the due effort. She was respectful of people's opinions.


In [ ]:
# Paragraph 4 - CCA_1

# Might need to include personal traits (see testimonial document)

CCA = "Students' Council" #@param {type:"string"}
role = "House Captain for the Moor-Tarbet House Directorate" #@param {type:"string"}

task_ = True #@param {type:"boolean"}
#@markdown "In his Co-curricular Activity (CCA), ..."
task = "her role was to oversee the entire CCA and made sure no one is slacking." #@param {type:"string"}

experience_ = True #@param {type:"boolean"}
#@markdown "During that time, ..." (Can include setbacks/experiences)
experience = "She planned multiple big events and worked with many groups. One notable event was planning Orientation. She messed up many times but came back stronger each time." #@param {type:"string"}

lesson_ = True #@param {type:"boolean"}
#@markdown "From his experiences, ..." (use learnt/honed/demonstrated)
lesson = "She learnt how to manage expectations and plan excellent events." #@param {type:"string"}

award_ = False #@param {type:"boolean"}
#@markdown "As a result of his perseverance and dedication, ..."
award = "" #@param {type:"string"}

paragraph_4 = f"{name} is{a_or_an(role)}{role} of {CCA}."

if task_ == True:
  paragraph_4 += f" In his Co-curricular Activity (CCA), {task}"
if experience_ == True:
  paragraph_4 += f" During that time, {experience}"
if lesson_ == True:
  paragraph_4 += f" From his experiences, {lesson}"
if award_ == True:
  paragraph_4 += f" As a result of his perseverance and dedication, {award}"

paragraph_4 = check_gender(paragraph_4, gender)

print(paragraph_4)

Jane is a House Captain for the Moor-Tarbet House Directorate of Students' Council. In her Co-curricular Activity (CCA), her role was to oversee the entire CCA and made sure no one is slacking. During that time, She planned multiple big events and worked with many groups. One notable event was planning Orientation. She messed up many times but came back stronger each time. From her experiences, She learnt how to manage expectations and plan excellent events.


In [ ]:
# Paragraph 5 - CCA_2

# Might need to include personal traits (see testimonial document)

has_second_cca = True #@param {type:"boolean"}

paragraph_5 = ""

if has_second_cca:
  CCA_2 = 'Photography Society' #@param {type:"string"}
  role = "member" #@param {type:"string"}

  task_ = True #@param {type:"boolean"}
  #@markdown "As part of (CCA), ..."
  task = "her role is to take photo of events." #@param {type:"string"}

  experience_ = True #@param {type:"boolean"}
  #@markdown "During that time, ..." (Can include setbacks/experiences)
  experience = "she struggled with capturing nice photos. But eventually with the help of her amazing teacher in charge, she succeeded." #@param {type:"string"}

  lesson_ = True #@param {type:"boolean"}
  #@markdown "Through his experiences, ..." (use learnt/honed/demonstrated)
  lesson = "she demonstrated extreme resilience and perseverance. " #@param {type:"string"}

  award_ = True #@param {type:"boolean"}
  #@markdown "As a result of his determination and resilience, ..."
  award = "she won the best photographer of the year from National Geographic!" #@param {type:"string"}


  paragraph_5 = f"Other than being a part of {CCA}, {name} was also part of {CCA_2}."

  if task_ == True:
    paragraph_5 += f" As part of {CCA_2}, {task}"
  if experience_ == True:
    paragraph_5 += f" During that time, {experience}"
  if lesson_ == True:
    paragraph_5 += f" Through his experiences, {lesson}"
  if award_ == True:
    paragraph_5 += f" As a result of his perseverance and dedication, {award}"

paragraph_5 = check_gender(paragraph_5, gender)

print(paragraph_5)

Other than being a part of Students' Council, Jane was also part of Photography Society. As part of Photography Society, her role is to take photo of events. During that time, she struggled with capturing nice photos. But eventually with the help of her amazing teacher in charge, she succeeded. Through her experiences, she demonstrated extreme resilience and perseverance.  As a result of her perseverance and dedication, she won the best photographer of the year from National Geographic!


In [ ]:
# Paragraph 6 - VIA_1

number_of_VIA = "2" #@param [0,1,2]

via_hours = "A lot" #@param ["A bit", "A lot"]

sample_passages = {
    "A bit":["<name> actively gives back to the community.", "<name> also participates actively in community service.", "Apart from his studies and involvement in his CCA, <name> set aside time to serve the community.", "<name> also found time to give back to the community amidst his busy schedule."],
    "A lot":["Strongly believing in having a heart for service and making a positive difference to others, <name> participated in many Values in Action (VIA) projects."]
}

# Student VIA information

#@markdown "John volunteered his services in the ..."
name_of_VIA_1 = 'Heartware Network' #@param {type:"string"}

#@markdown "In his volunteering stint, ..." (Include experiences)
experience_1 = "her job was to teach primary school students tuition." #@param {type:"string"}

#@markdown "Through this experience, ..." (use learnt/honed/demonstrated)
lesson_1 = "she learnt that she needs to be patient." #@param {type:"string"}

#@markdown "Additionally, John participated in the Values in Action (VIA) project where he worked with..."
name_of_VIA_2 = 'the National University of Singapore (NUS)' #@param {type:"string"}

#@markdown "As part of the programme, ..." (Include experiences)
experience_2 = "she was tasked to look after the school compound in Yale-NUS" #@param {type:"string"}

#@markdown "From his experiences, ..." (use learnt/honed/demonstrated)
lesson_2 = "she learnt how difficult it is to ensure that everyone in Yale-NUS follows rules." #@param {type:"string"}

# Generative text
intro_1 = f" He volunteered his services with {name_of_VIA_1}."
intro_2 = f" Additionally, <name> participated in the Values in Action (VIA) project where he worked with {name_of_VIA_2}."

# Start paragraph 6
len_sample_chosen = len(sample_passages[via_hours])-1

paragraph_6_start = sample_passages[via_hours][random.randint(0,len_sample_chosen)]

paragraph_6 = "" 

if number_of_VIA == "0":
  paragraph_6 = "" # No VIA at all, so nothing on this
elif number_of_VIA == "1":
  experience_1_passage = f" In his volunteering stint, {experience_1}"
  lesson_1_passage = f" Through this experience, {lesson_1}"
  paragraph_6 = paragraph_6_start + intro_1 + experience_1_passage + lesson_1_passage
else:
  # First VIA experience
  experience_1_passage = f" In his volunteering stint, {experience_1}"
  lesson_1_passage = f" Through this experience, {lesson_1}"
  paragraph_6 = paragraph_6_start + intro_1 + experience_1_passage + lesson_1_passage
  
  # Second VIA experience
  experience_2_passage = f" As part of the programme, {experience_2}"
  lesson_2_passage = f" From his experiences, {lesson_2}"
  paragraph_6 = paragraph_6 + intro_2 + experience_2_passage + lesson_2_passage


paragraph_6 = paragraph_6.replace('<name>',name)

paragraph_6 = check_gender(paragraph_6, gender)

print(paragraph_6)

Strongly believing in having a heart for service and making a positive difference to others, Jane participated in many Values in Action (VIA) projects. She volunteered her services with Heartware Network. In her volunteering stint, her job was to teach primary school students tuition. Through this experience, she learnt that she needs to be patient. Additionally, Jane participated in the Values in Action (VIA) project where she worked with the National University of Singapore (NUS). As part of the programme, she was tasked to look after the school compound in Yale-NUS From her experiences, she learnt how difficult it is to ensure that everyone in Yale-NUS follows rules.


In [ ]:
# Paragraph 7 - Explored out

explored_out = True #@param {type:"boolean"}

paragraph_7 = ""

if explored_out:

  intro_array = [
      "Outside of school, <name> actively sought out opportunities to widen his horizons beyond his passion.",
      "<name> is also open-minded and curious to try new things. Outside of school, ",
  ]

  #@markdown "E.g. She had a job shadowing opportunity at the SengKang General Hospital."
  what_was_it = 'Shee had a job shadowing opportunity at the SengKang General Hospital.' #@param {type:"string"}

  #@markdown "In that time, ..." (Include experiences)
  experience = "her job was to help buy coffee for doctors." #@param {type:"string"}

  #@markdown "Through this experience, ..." (use learnt/honed/demonstrated)
  lesson = "she demonstrated much humility. " #@param {type:"string"}

  what_was_it = " " + what_was_it
  experience_passage = f" In that time, {experience}"
  lesson_passage = f" Through this experience, {lesson}"

  paragraph_7 = intro_array[random.randint(0,len(intro_array)-1)] + what_was_it + experience_passage + lesson_passage

paragraph_7 = paragraph_7.replace('<name>',name)

paragraph_7 = check_gender(paragraph_7, gender)

print(paragraph_7)

Jane is also open-minded and curious to try new things. Outside of school,  Shee had a job shadowing opportunity at the SengKang General Hospital. In that time, her job was to help buy coffee for doctors. Through this experience, she demonstrated much humility. 


In [ ]:
# Paragraph 8 - Concluding statements

random_statements = [
    "In all, <name> is an individual of outstanding character and values. He is a passionate and fun-loving person with a positive outlook on life. Even in the face of setbacks, he is resilient and determined to overcome any obstacles in his way. With his high emotional quotient and fortitude, <name> is poised for success in his future endeavours and will be an asset to any organisation she chooses to join.",
    "A promising student with an inquisitive mind, <name> has demonstrated resilience and adaptability in pursuing his endeavours and has the potential to achieve whatever he sets out to do in the future.",
    "<name> has shown himself to be an amiable student man with a positive outlook on life. His peers see him as a supportive and trustworthy friend who often puts them before himself and shows genuine care and concern for them. He is a good-natured and confident student with much potential and will certainly achieve greater heights in his future endeavours.",
    "<name> is an agreeable character but he also has a strong innate determination to see things through. He has a positive and open mindset that rubs off on others around him, while also having the intellectual humility to learn from others. He is steadfast, does not falter in adverse conditions, and can be counted on to handle challenging and stressful situations well. He is a promising young man who deserves every opportunity to achieve his potential. He will be an asset to his community.",
    "<name> possesses an inquisitive mind with a genuine intellectual liveliness. He is an outspoken individual who is not afraid to voice his opinions. He displays strong passion and commitment in his areas of interests. He is willing to work hard to achieve his goals and will persevere until he succeeds. <name>’s positive traits will undoubtedly stand him in good stead in his future endeavours."
]

paragraph_8 = random_statements[random.randint(0,len(random_statements)-1)]

paragraph_8 = paragraph_8.replace('<name>',name)

paragraph_8 = check_gender(paragraph_8, gender)

print(paragraph_8)

In all, Jane is an individual of outstanding character and values. She is a passionate and fun-loving person with a positive outlook on life. Even in the face of setbacks, she is resilient and determined to overcome any obstacles in her way. With her high emotional quotient and fortitude, Jane is poised for success in her future endeavours and will be an asset to any organisation she chooses to join.


# Combine all

In [ ]:
# Combine all

full_testimonial_array = []

for paragraph in [paragraph_1, paragraph_2, paragraph_3, paragraph_4, paragraph_5, paragraph_6, paragraph_7, paragraph_8]:
  if paragraph != "":
    full_testimonial_array.append(paragraph)

full_testimonial = '\n\n'.join(full_testimonial_array)

print(full_testimonial)

number_of_words = len(full_testimonial.split())

print("")
print("")
print(f'(Number of words: {number_of_words})')

Jane is a hardworking, self-directed and focused individual and has consistently demonstrated exemplary attitude and self-motivation. She is a respectful student who actively participates in class discussions. She has an amiable personality and is well-liked by her peers. She was quite exemplary in her behaviour and attitude and thus she was a recipient of the school's F.I.R.E. award - which is for students who are role models and display the school's core values of fortitude, integrity, respect and enterprise.

Academically, Jane was a consistent and strong performer. Her ability to reflect on her learning and keep pace with the rigorous school curriculum were key to her academic success. As an independent learner, Jane took a systematic and proactive approach to monitoring her own progress. She developed strong cognitive and analytical skills over the two years and impressed her teachers with her dedication to her academic responsibilities.

As a member of her Project Work group, she

In [29]:
# Cell to save as MS word

from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Create new document
document = Document()

# Add text
paragraph = document.add_paragraph(full_text)

# Change style
style = document.styles['Normal']
font = style.font
font.name = 'Arial'
font.size = Pt(11)

# align paragraph
paragraph.alignment = WD_ALIGN_PARAGRAPH.JUSTIFY

document.save(f'./doc/{name}.docx')

In [ ]:
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Download All")
output = widgets.Output()

button_all = widgets.Button(description="Download Current")
output_all = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    # Save all documents as zip
    !zip -r ./testimonials.zip ./doc

    from google.colab import files
    files.download("./testimonials.zip")

def on_button_all_clicked(b):
  # Display the message within the output widget.
  with output_all:
    # Save current document
    from google.colab import files
    files.download(f"./doc/{name}.docx")

button.on_click(on_button_clicked)
display(button, output)

button_all.on_click(on_button_all_clicked)
display(button_all, output_all)

# To-do
1. Do gender check (v)
2. Paraphrase (on hold)
3. Combine all (v)
4. Save to downloadable MS Word (v)
5. Use Grammarly on MS Word
6. Add more functionalities
- A few more phrases from each category
- Class rep/ subject rep
- OGL

In [ ]:
# Utilities 
# Forms: https://colab.research.google.com/notebooks/forms.ipynb#scrollTo=3jKM6GfzlgpS